# IMPORTS

In [11]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# in set so duplicate get dissolved
stop_words = set(stopwords.words('english'))
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report , ConfusionMatrixDisplay ,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, datasets
import pickle

In [4]:
#Read CSV
# source Kaggle : 
#https://www.kaggle.com/datasets/sonalgarg174/ifnd-dataset

news_df = pd.read_csv('IFND.csv', encoding='latin1')
news_df.head()

,id,Statement,Image,Web,Category,Date,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
1,3,"In Delhi, Deputy US Secretary of State Stephen...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,VIOLENCE,Oct-20,TRUE
2,4,LAC tensions: China's strategy behind delibera...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,TERROR,Oct-20,TRUE
3,5,India has signed 250 documents on Space cooper...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
4,6,Tamil Nadu chief minister's mother passes away...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,ELECTION,Oct-20,TRUE


# preprocessing

In [ ]:
#drop not needed values

news_df.drop('Date' ,axis=1,inplace=True)
news_df.drop('Image' ,axis=1,inplace=True)
news_df.head()

In [65]:
# encoding label and category to int
label_encoder = LabelEncoder()
news_df['Label'] = label_encoder.fit_transform(news_df['Label'] )
news_df['Web'] = label_encoder.fit_transform(news_df['Web'] )
news_df['Category'] = label_encoder.fit_transform(news_df['Category'] )
news_df

,id,Statement,Web,Category,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",8,0,1
1,3,"In Delhi, Deputy US Secretary of State Stephen...",8,8,1
2,4,LAC tensions: China's strategy behind delibera...,8,6,1
3,5,India has signed 250 documents on Space cooper...,8,0,1
4,6,Tamil Nadu chief minister's mother passes away...,8,1,1
...,...,...,...,...,...
56709,56711,Fact Check: This is not Bruce Lee playing ping...,14,4,0
56710,56712,Fact Check: Did Japan construct this bridge in...,14,0,0
56711,56713,Fact Check: Viral video of Mexico earthquake i...,14,4,0
56712,56714,Fact Check: Ballet performance by Chinese coup...,14,0,0


In [67]:
#shuffle the datset and save the dataset because intials all are true and then all are false
shuffled_df = news_df.sample(frac=1, random_state=42)
shuffled_df.to_csv('shuffled_dataset.csv', index=False)

In [5]:
# read the shuffeled saved dataset from the disk

shuffled_df2 = pd.read_csv('shuffled_dataset.csv')
shuffled_df2

,id,Statement,Web,Category,Label
0,25355,The man on the phone: What's it like making hi...,5,5,1
1,4093,PM to states: start planning for vaccine rollo...,13,0,1
2,28348,DU survey shows Akhilesh-Maya more popular tha...,23,1,1
3,16650,UP registers first case under anti-conversion ...,27,6,1
4,41229,Fact Check: Old Video Of Statues Of Ganesha Be...,21,8,0
...,...,...,...,...,...
56709,54345,Fact Checked: This picture is not from Hizbul ...,2,4,0
56710,38160,Fact Check: Viral photo of the flood is not fr...,14,0,0
56711,862,Sushant Singh Rajput case: Û÷Uddhav never int...,13,8,1
56712,15797,"11 migrants on way home die, 14 others injured...",27,2,1


In [4]:
shuffled_df2.drop('id' ,axis=1,inplace=True)
shuffled_df2

,Statement,Web,Category,Label
0,The man on the phone: What's it like making hi...,5,5,1
1,PM to states: start planning for vaccine rollo...,13,0,1
2,DU survey shows Akhilesh-Maya more popular tha...,23,1,1
3,UP registers first case under anti-conversion ...,27,6,1
4,Fact Check: Old Video Of Statues Of Ganesha Be...,21,8,0
...,...,...,...,...
56709,Fact Checked: This picture is not from Hizbul ...,2,4,0
56710,Fact Check: Viral photo of the flood is not fr...,14,0,0
56711,Sushant Singh Rajput case: Û÷Uddhav never int...,13,8,1
56712,"11 migrants on way home die, 14 others injured...",27,2,1


## cleaning data set nltk removing stopwords and stemming


In [11]:
# data preprocessing removing all except alphabets then stemming them

stemmer = PorterStemmer() 
def data_processing(clean_text): 
    #print(type(clean_text))
    clean_text = clean_text.lower()
    clean_text = re.sub("[^a-zA-Z]"," ",clean_text)
    text_tokens = word_tokenize(clean_text)
    #filtered_text = [w for w in text_tokens if not w in stop_words]
    filtered_words = []
    string = ""
    for word in text_tokens:
        if word not in stop_words:
            word = stemmer.stem(word)
            string = string+" "+word
            
    return string


In [12]:
shuffled_df2['Statement'] = shuffled_df2['Statement'].apply(data_processing)
stemmed_processed_dataset = shuffled_df2
shuffled_df2

,id,Statement,Web,Category,Label
0,25355,man phone like make histori highest auction bid,5,5,1
1,4093,pm state start plan vaccin rollout maintain f...,13,0,1
2,28348,du survey show akhilesh maya popular modi pri...,23,1,1
3,16650,regist first case anti convers law bareilli d...,27,6,1
4,41229,fact check old video statu ganesha immers riv...,21,8,0
...,...,...,...,...,...
56709,54345,fact check pictur hizbul command zakir musa f...,2,4,0
56710,38160,fact check viral photo flood bihar australia,14,0,0
56711,862,sushant singh rajput case uddhav never interf...,13,8,1
56712,15797,migrant way home die other injur separ incid mp,27,2,1


In [13]:
#drop web coulumn because we dont necesrily have to incorporate it / usualy when testing in real world this data is hard to get
shuffled_df2.drop('Web',axis=1,inplace=True)
shuffled_df2

,id,Statement,Category,Label
0,25355,man phone like make histori highest auction bid,5,1
1,4093,pm state start plan vaccin rollout maintain f...,0,1
2,28348,du survey show akhilesh maya popular modi pri...,1,1
3,16650,regist first case anti convers law bareilli d...,6,1
4,41229,fact check old video statu ganesha immers riv...,8,0
...,...,...,...,...
56709,54345,fact check pictur hizbul command zakir musa f...,4,0
56710,38160,fact check viral photo flood bihar australia,0,0
56711,862,sushant singh rajput case uddhav never interf...,8,1
56712,15797,migrant way home die other injur separ incid mp,2,1


# TFIDF converting words to number for ML algorithms

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
sample_size = int(0.5 * len(shuffled_df2))

# Randomly sample the data
downsample_random = shuffled_df2.sample(n=sample_size, random_state=42)
downsample_random

,id,Statement,Category,Label
25353,50351,video chhattisgarh share teacher rm vm school...,8,0
4091,13747,rjd aap protest exclus pm parti meet lac stan...,1,1
28346,36704,delhi taxi driver arrest shoot bulandshahr cop,1,1
16648,43747,fake news chao whatsapp roll featur highlight...,4,0
41227,45183,bjp promis free covid vaccin everi bihar resi...,0,0
...,...,...,...,...
4490,4641,sunni write capt blame govt fan farmer protest,2,1
52558,36306,mayawati expect mahagathbandhan prospect impr...,2,1
3978,47851,unrel video bihar share proof polic brutal,8,0
11051,29408,ram templ nation issu haryana cm manohar lal ...,2,1


In [16]:
tiv = TfidfVectorizer(max_features=4500)
X_train , X_test , y_train , y_test = train_test_split(downsample_random['Statement'],downsample_random['Label'], test_size = 0.30 ,random_state=42)


x_train = tiv.fit_transform(X_train).toarray()
x_test = tiv.transform(X_test).toarray()
print(x_train.shape)

(19849, 4500)


In [18]:

print(y_train.shape)
for i in x_train[600]:
    if(i>0):
        print(i)

(39699,)
0.4877130008630547
0.31863779677377585
0.32877853149844133
0.2172353967756807
0.28411474157308225
0.2169053856127736
0.22687127113866623
0.31365825082960297
0.31365825082960297
0.35960636797439666


# Training on ML models

### Logistic regression

In [13]:
#log
logistic_regression = LogisticRegression()

# Define hyperparameter grid for tuning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'penalty': ['l1', 'l2'],  # Solver algorithm
}

# Create a GridSearchCV object to find the best hyperparameters
grid_search = GridSearchCV(estimator=logistic_regression, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(x_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_


# Evaluate the model on the testing data
y_pred = best_model.predict(x_test)

# Print classification report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 24 candidates, totalling 120 fits


D:\00Installation\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "D:\00Installation\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\00Installation\Anaconda\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "D:\00Installation\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = _check_solver(self.solver, self.penalty, s

Best Hyperparameters: {'C': 10, 'max_iter': 200, 'penalty': 'l2'}
              precision    recall  f1-score   support

           0       0.94      0.85      0.89      2859
           1       0.93      0.97      0.95      5649

    accuracy                           0.93      8508
   macro avg       0.93      0.91      0.92      8508
weighted avg       0.93      0.93      0.93      8508



## Naive Bayes

In [20]:
# naive bayes lgorithm

from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# Create a Multinomial Naive Bayes classifier
naive_bayes = MultinomialNB()

# Define hyperparameter grid for tuning
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0],  # Smoothing parameter (Laplace smoothing)
}

# Create a GridSearchCV object to find the best hyperparameters
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(x_train, y_train)

# Get the best hyperparameters
best_paramsNB = grid_search.best_params_
print("Best Hyperparameters:", best_paramsNB)

# Get the best model
best_modelNB = grid_search.best_estimator_

# Evaluate the model on the testing data
y_predNB = best_modelNB.predict(x_test)

# Print classification report
print(classification_report(y_test, y_predNB))


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Hyperparameters: {'alpha': 1.0}
              precision    recall  f1-score   support

           0       0.94      0.83      0.88      2859
           1       0.92      0.97      0.94      5649

    accuracy                           0.92      8508
   macro avg       0.93      0.90      0.91      8508
weighted avg       0.92      0.92      0.92      8508



## saving models 

In [21]:
with open('Naive_Bayes','wb') as f:
    pickle.dump(best_modelNB,f)
    

In [ ]:
with open('logisticRegression_model_pickle','wb') as f:
    pickle.dump(best_model,f)
    

In [ ]:
# Save the TfidfVectorizer and transformed data to a file

with open('tfidf_vectorizer', 'wb') as vectorizer_file:
    pickle.dump(tiv, vectorizer_file)


### open the mode for testing 

In [12]:
#opening the models

with open('logisticRegression_model_pickle','rb')as f:
    LogR = pickle.load(f)
with open('Naive_Bayes','rb')as f:
    NB = pickle.load(f)

In [13]:
# Later, when you want to use them again, you can load them from the saved files

with open('tfidf_vectorizer', 'rb') as vectorizer_file:
    loaded_tfidf_vectorizer = pickle.load(vectorizer_file)


## Testing

###### sample news to test

###### True
The man on the phone: What's it like making history's highest auction bid? 

In reply to Congress, FB says it is non-partisan, denounces hate

Less than 45% of households in five states use clean fuel for cooking: Government survey

No coercive action against private companies for non-payment of wages during COVID-19: SC



###### false
Fact Check: Old Video Of Statues Of Ganesha Being Immersed Into River Falsely Linked With

Animation Video Shared As Mermaid 

No, This Video Does Not Show Soldiers Celebrating Ganpati At Galwan...

Fake: Passed off as Bengal violence, these images are from Pakistan 

Viral video showing cracker stalls gutted during Diwali is not from Khammam

In [25]:
# pre processing for input data

stemmer = PorterStemmer() 
def pre_process(clean_text): 
    #print(type(clean_text))
    clean_text = clean_text.lower()
    clean_text = re.sub("[^a-zA-Z]"," ",clean_text)
    text_tokens = word_tokenize(clean_text)
    #filtered_text = [w for w in text_tokens if not w in stop_words]
    filtered_words = []
    string = ""
    for word in text_tokens:
        if word not in stop_words:
            word = stemmer.stem(word)
            string = string+" "+word
            
    return string

# take input and processe it

news =input("enter the news")
news=pre_process(news) 
news = [" ".join(news.split())]
news_vector=loaded_tfidf_vectorizer.transform(news).toarray()
resutNB = NB.predict(news_vector)
resultLR =LogR.predict(news_vector)
print(news)

enter the newsAnimation Video Shared As Mermaid
['anim video share mermaid']


In [26]:
if(resutNB==1):
    print( "True from Naive Bayes")
if(resutNB==0):
    print( "false from Naive Bayes")
    
if(resultLR==1):
    print( "True from Logistic Regression")
if(resultLR==0):
    print( "false from Logistic Regression")


false from Naive Bayes
false from Logistic Regression


## END